In [6]:
print(plt.style.available)


['Solarize_Light2', '_classic_test_patch', '_mpl-gallery', '_mpl-gallery-nogrid', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn-v0_8', 'seaborn-v0_8-bright', 'seaborn-v0_8-colorblind', 'seaborn-v0_8-dark', 'seaborn-v0_8-dark-palette', 'seaborn-v0_8-darkgrid', 'seaborn-v0_8-deep', 'seaborn-v0_8-muted', 'seaborn-v0_8-notebook', 'seaborn-v0_8-paper', 'seaborn-v0_8-pastel', 'seaborn-v0_8-poster', 'seaborn-v0_8-talk', 'seaborn-v0_8-ticks', 'seaborn-v0_8-white', 'seaborn-v0_8-whitegrid', 'tableau-colorblind10']


In [9]:
# %% [markdown]
# # Simulación y Ajuste de Modelos Dinámicos en Python
# 
# Este notebook reproduce ejemplos de simulación similares al script en R (usando grindR) e incluye:
# 
# - **Modelo Predador-Presa (ODE)** con opción de evento y adición de ruido gaussiano.
# - **Mapa Discreto (Logístico)**.
# - **Ecuaciones Diferenciales con Retardo (DDE)** para un modelo Lotka–Volterra modificado.
# - **Modelo con Múltiples Especies** (vectores de ecuaciones).
# - **Ajuste de Parámetros** a datos simulados.
# 
# Se utilizan sliders (ipywidgets) para modificar los parámetros y visualizar interactivamente el comportamiento de cada sistema.

# %%
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.optimize import least_squares
import ipywidgets as widgets
from ipywidgets import interact

# Para que los gráficos se muestren en el notebook
%matplotlib inline
#plt.style.use('seaborn-v0_8-dark')
plt.style.use('Solarize_Light2')

# %% [markdown]
# ## 1. Modelo Predador-Presa (ODE) interactivo
# 
# El sistema se define como:
# 
# \\[
# \\frac{dP}{dt} = \\alpha P - \\beta P D \\quad\\text{y}\\quad \\frac{dD}{dt} = \\gamma P D - \\delta D
# \\]
# 
# Se incluye la opción de aplicar un evento en \\(t=20\\) para fijar la población de depredadores (D) a 0.

# %%
def predator_prey_ode(t, y, alpha, beta, gamma, delta):
    P, D = y
    dP = alpha * P - beta * P * D
    dD = gamma * P * D - delta * D
    return [dP, dD]

def simulate_predator_prey(alpha=0.67, beta=1.33, gamma=1.0, delta=1.0, 
                             P0=1.0, D0=1.0, t_max=25, event_reset=False):
    if event_reset and t_max > 20:
        # Simulación desde t=0 hasta t=20
        sol1 = solve_ivp(lambda t, y: predator_prey_ode(t, y, alpha, beta, gamma, delta),
                         [0, 20], [P0, D0], t_eval=np.linspace(0, 20, 201))
        # Al llegar a t=20 se fija D=0
        P_event = sol1.y[0, -1]
        y_event = [P_event, 0.0]
        # Simulación desde t=20 hasta t_max
        sol2 = solve_ivp(lambda t, y: predator_prey_ode(t, y, alpha, beta, gamma, delta),
                         [20, t_max], y_event, t_eval=np.linspace(20, t_max, int((t_max-20)*20)+1))
        # Unir ambas soluciones
        t_full = np.concatenate((sol1.t, sol2.t))
        P_full = np.concatenate((sol1.y[0], sol2.y[0]))
        D_full = np.concatenate((sol1.y[1], sol2.y[1]))
    else:
        sol = solve_ivp(lambda t, y: predator_prey_ode(t, y, alpha, beta, gamma, delta),
                        [0, t_max], [P0, D0], t_eval=np.linspace(0, t_max, int(t_max*20)+1))
        t_full = sol.t
        P_full = sol.y[0]
        D_full = sol.y[1]
    
    plt.figure(figsize=(8, 4))
    plt.plot(t_full, P_full, label='Prey (P)')
    plt.plot(t_full, D_full, label='Predator (D)')
    plt.xlabel('Time')
    plt.ylabel('Population')
    plt.title('Modelo Predador-Presa')
    plt.legend()
    plt.show()

# Crear el widget interactivo para el modelo ODE
interact(simulate_predator_prey,
         alpha=widgets.FloatSlider(min=0.1, max=2.0, step=0.1, value=0.67, description='alpha'),
         beta=widgets.FloatSlider(min=0.1, max=3.0, step=0.1, value=1.33, description='beta'),
         gamma=widgets.FloatSlider(min=0.1, max=3.0, step=0.1, value=1.0, description='gamma'),
         delta=widgets.FloatSlider(min=0.1, max=3.0, step=0.1, value=1.0, description='delta'),
         P0=widgets.FloatSlider(min=0.1, max=5.0, step=0.1, value=1.0, description='P0'),
         D0=widgets.FloatSlider(min=0.1, max=5.0, step=0.1, value=1.0, description='D0'),
         t_max=widgets.IntSlider(min=10, max=50, step=1, value=25, description='t_max'),
         event_reset=widgets.Checkbox(value=False, description='Reset D @ t=20')
        )

# %% [markdown]
# ## 1.b Modelo Predador-Presa con Ruido (Euler + ruido gaussiano)
# 
# Se simula el mismo modelo usando el método de Euler y se añade ruido gaussiano en cada paso.

# %%
def predator_prey_euler(alpha=0.67, beta=1.33, gamma=1.0, delta=1.0, 
                          P0=1.0, D0=1.0, t_max=25, dt=0.01, noise_sd=0.01):
    n_steps = int(t_max/dt) + 1
    t = np.linspace(0, t_max, n_steps)
    P = np.zeros(n_steps)
    D = np.zeros(n_steps)
    P[0] = P0
    D[0] = D0
    for i in range(n_steps - 1):
        dP = alpha * P[i] - beta * P[i] * D[i]
        dD = gamma * P[i] * D[i] - delta * D[i]
        # Método de Euler + adición de ruido gaussiano
        P[i+1] = P[i] + dt * dP + np.random.normal(0, noise_sd)
        D[i+1] = D[i] + dt * dD + np.random.normal(0, noise_sd)
    
    plt.figure(figsize=(8, 4))
    plt.plot(t, P, label='Prey (P)')
    plt.plot(t, D, label='Predator (D)')
    plt.xlabel('Time')
    plt.ylabel('Population')
    plt.title('Modelo Predador-Presa con Ruido')
    plt.legend()
    plt.show()

interact(predator_prey_euler,
         alpha=widgets.FloatSlider(min=0.1, max=2.0, step=0.1, value=0.67, description='alpha'),
         beta=widgets.FloatSlider(min=0.1, max=3.0, step=0.1, value=1.33, description='beta'),
         gamma=widgets.FloatSlider(min=0.1, max=3.0, step=0.1, value=1.0, description='gamma'),
         delta=widgets.FloatSlider(min=0.1, max=3.0, step=0.1, value=1.0, description='delta'),
         P0=widgets.FloatSlider(min=0.1, max=5.0, step=0.1, value=1.0, description='P0'),
         D0=widgets.FloatSlider(min=0.1, max=5.0, step=0.1, value=1.0, description='D0'),
         t_max=widgets.IntSlider(min=10, max=50, step=1, value=25, description='t_max'),
         dt=widgets.FloatSlider(min=0.001, max=0.1, step=0.001, value=0.01, description='dt'),
         noise_sd=widgets.FloatSlider(min=0.0, max=0.1, step=0.001, value=0.01, description='noise_sd')
        )

# %% [markdown]
# ## 2. Mapa Discreto (Logístico)
# 
# El mapa discreto se define como:
# 
# \\[
# N_{t+1} = r \\cdot N_t (1 - N_t)
# \\]
# 
# Se simulan 1000 iteraciones y se grafica un diagrama de dispersión de \\(N_t\\) vs \\(N_{t+1}\\).

# %%
def discrete_map(r=3.75, N0=0.01, n_iter=1000):
    N = np.zeros(n_iter)
    N[0] = N0
    for t in range(n_iter - 1):
        N[t+1] = r * N[t] * (1 - N[t])
    
    plt.figure(figsize=(6, 5))
    plt.scatter(N[:-1], N[1:], s=1, color='blue')
    plt.xlabel('$N_t$')
    plt.ylabel('$N_{t+1}$')
    plt.title('Diagrama de Dispersión del Mapa Logístico')
    plt.show()

interact(discrete_map,
         r=widgets.FloatSlider(min=0.1, max=4.5, step=0.1, value=3.75, description='r'),
         N0=widgets.FloatSlider(min=0.001, max=0.1, step=0.001, value=0.01, description='N0'),
         n_iter=widgets.IntSlider(min=100, max=2000, step=50, value=1000, description='Iteraciones')
        )

# %% [markdown]
# ## 3. Ecuaciones Diferenciales con Retardo (DDE) para Lotka–Volterra
# 
# Se implementa un solver sencillo (tipo Euler) para un DDE. El modelo es:
# 
# \\[
# \\frac{dR}{dt} = r R (1 - R/K) - a R N \\quad \\frac{dN}{dt} = a R_{lag} N_{lag} - d N
# \\]
# 
# donde \\(R_{lag}\\) y \\(N_{lag}\\) son los valores en \\(t - \\Delta\\); si \\(t-\\Delta < 0\\), se usan 0.

# %%
def dde_solver(r=1.0, K=1.0, a=1.0, d=0.5, Delta=10, R0=1.0, N0=0.1, t_max=50, dt=0.01):
    n_steps = int(t_max/dt) + 1
    t_vals = np.linspace(0, t_max, n_steps)
    R = np.zeros(n_steps)
    N = np.zeros(n_steps)
    R[0] = R0
    N[0] = N0
    
    for i in range(n_steps - 1):
        t = t_vals[i]
        t_lag = t - Delta
        if t_lag < 0:
            R_lag = 0.0
            N_lag = 0.0
        else:
            R_lag = np.interp(t_lag, t_vals[:i+1], R[:i+1])
            N_lag = np.interp(t_lag, t_vals[:i+1], N[:i+1])
        dR = r * R[i] * (1 - R[i]/K) - a * R[i] * N[i]
        dN = a * R_lag * N_lag - d * N[i]
        R[i+1] = R[i] + dt * dR
        N[i+1] = N[i] + dt * dN
    
    plt.figure(figsize=(8, 4))
    plt.plot(t_vals, R, label='R (Prey)')
    plt.plot(t_vals, N, label='N (Predator)')
    plt.xlabel('Time')
    plt.ylabel('Population')
    plt.title('DDE: Lotka–Volterra con retardo')
    plt.legend()
    plt.show()

interact(dde_solver,
         r=widgets.FloatSlider(min=0.1, max=3.0, step=0.1, value=1.0, description='r'),
         K=widgets.FloatSlider(min=0.5, max=5.0, step=0.1, value=1.0, description='K'),
         a=widgets.FloatSlider(min=0.1, max=3.0, step=0.1, value=1.0, description='a'),
         d=widgets.FloatSlider(min=0.1, max=2.0, step=0.1, value=0.5, description='d'),
         Delta=widgets.IntSlider(min=1, max=20, step=1, value=10, description='Delta'),
         R0=widgets.FloatSlider(min=0.1, max=2.0, step=0.1, value=1.0, description='R0'),
         N0=widgets.FloatSlider(min=0.01, max=1.0, step=0.01, value=0.1, description='N0'),
         t_max=widgets.IntSlider(min=20, max=100, step=5, value=50, description='t_max'),
         dt=widgets.FloatSlider(min=0.001, max=0.05, step=0.001, value=0.01, description='dt')
        )

# %% [markdown]
# ## 4. Modelo con Múltiples Especies (n = 3)
# 
# Se modela la competencia entre 3 poblaciones de presas (R₁, R₂, R₃) y sus respectivos depredadores (N₁, N₂, N₃).
# 
# Las ecuaciones son:
# 
# \\[
# \\frac{dR_i}{dt} = b_i R_i (1 - S) - d1 R_i - a R_i N_i \\quad \\text{con } S = \\sum_{i=1}^n R_i
# \\]
# 
# \\[
# \\frac{dN_i}{dt} = a R_i N_i - d2 N_i
# \\]
# 
# donde los coeficientes \\(b_i\\) se generan aleatoriamente.

# %%
def multi_species(n=3, d1=0.1, d2=0.2, a=1.0, t_max=50):
    # Generar parámetros b aleatorios para cada especie
    np.random.seed(0)
    b = np.random.normal(1.0, 0.1, n)
    
    # Condiciones iniciales para R y N
    R0 = np.full(n, 0.1/n)
    N0 = np.full(n, 0.01/n)
    state0 = np.concatenate((R0, N0))
    
    def multi_ode(t, y):
        R = y[:n]
        N = y[n:]
        S = np.sum(R)
        dR = b * R * (1 - S) - d1 * R - a * R * N
        dN = a * R * N - d2 * N
        return np.concatenate((dR, dN))
    
    sol = solve_ivp(multi_ode, [0, t_max], state0, t_eval=np.linspace(0, t_max, int(t_max*20)+1))
    t = sol.t
    R_sol = sol.y[:n, :]
    N_sol = sol.y[n:, :]
    
    fig, axs = plt.subplots(2, 1, figsize=(8, 8))
    for i in range(n):
        axs[0].plot(t, R_sol[i], label=f'R{i+1}')
        axs[1].plot(t, N_sol[i], label=f'N{i+1}')
    axs[0].set_title('Poblaciones de Presas (R)')
    axs[1].set_title('Poblaciones de Depredadores (N)')
    for ax in axs:
        ax.set_xlabel('Time')
        ax.set_ylabel('Population')
        ax.legend()
    plt.tight_layout()
    plt.show()

interact(multi_species,
         n=widgets.IntSlider(min=2, max=6, step=1, value=3, description='n'),
         d1=widgets.FloatSlider(min=0.01, max=0.5, step=0.01, value=0.1, description='d1'),
         d2=widgets.FloatSlider(min=0.01, max=0.5, step=0.01, value=0.2, description='d2'),
         a=widgets.FloatSlider(min=0.1, max=3.0, step=0.1, value=1.0, description='a'),
         t_max=widgets.IntSlider(min=20, max=100, step=5, value=50, description='t_max')
        )

# %% [markdown]
# ## 5. Ajuste de Parámetros
# 
# Se simula un conjunto de datos usando el modelo predador-presa y se utiliza una optimización de mínimos cuadrados para ajustar los parámetros del siguiente modelo:
# 
# \\[
# \\frac{dR}{dt} = r R (1 - R/K) - a R N \\quad \\frac{dN}{dt} = c \\cdot a R N - \\delta N
# \\]
# 
# Se comparan la solución ajustada y los datos simulados.

# %%
def predator_prey_model(t, y, r, K, a, c, delta):
    R, N = y
    dR = r * R * (1 - R/K) - a * R * N
    dN = c * a * R * N - delta * N
    return [dR, dN]

def simulate_model(params, y0, t_span):
    r, K, a, c, delta = params
    sol = solve_ivp(lambda t, y: predator_prey_model(t, y, r, K, a, c, delta),
                    t_span, y0, t_eval=np.linspace(t_span[0], t_span[1], 200))
    return sol.t, sol.y

# Generar datos simulados (modelo base)
true_params = [1.0, 1.0, 1.0, 1.0, 0.5]  # [r, K, a, c, delta]
y0 = [1.0, 0.01]
t_span = [0, 20]
t_data, y_data = simulate_model(true_params, y0, t_span)

# Agregar ruido a los datos
np.random.seed(1)
R_data = y_data[0] + np.random.normal(0, 0.02, len(t_data))
N_data = y_data[1] + np.random.normal(0, 0.002, len(t_data))

def residuals(p):
    t_sim, y_sim = simulate_model(p, y0, t_span)
    R_sim = np.interp(t_data, t_sim, y_sim[0])
    N_sim = np.interp(t_data, t_sim, y_sim[1])
    res = np.concatenate((R_sim - R_data, N_sim - N_data))
    return res

def fit_model(r_init=1.0, K_init=1.0, a_init=1.0, c_init=1.0, delta_init=0.5):
    p0 = [r_init, K_init, a_init, c_init, delta_init]
    res = least_squares(residuals, p0, bounds=([0,0,0,0,0], [5,5,5,5,5]))
    fitted_params = res.x
    t_fit, y_fit = simulate_model(fitted_params, y0, t_span)
    
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.plot(t_data, R_data, 'o', label='Datos R')
    plt.plot(t_fit, y_fit[0], '-', label='Ajuste R')
    plt.xlabel('Time')
    plt.ylabel('R')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(t_data, N_data, 'o', label='Datos N')
    plt.plot(t_fit, y_fit[1], '-', label='Ajuste N')
    plt.xlabel('Time')
    plt.ylabel('N')
    plt.legend()
    
    plt.suptitle('Ajuste de Parámetros')
    plt.show()
    
    print('Parámetros ajustados:')
    print(f'r = {fitted_params[0]:.3f}, K = {fitted_params[1]:.3f}, a = {fitted_params[2]:.3f}, c = {fitted_params[3]:.3f}, delta = {fitted_params[4]:.3f}')

interact(fit_model,
         r_init=widgets.FloatSlider(min=0.5, max=2.0, step=0.1, value=1.0, description='r'),
         K_init=widgets.FloatSlider(min=0.5, max=2.0, step=0.1, value=1.0, description='K'),
         a_init=widgets.FloatSlider(min=0.5, max=2.0, step=0.1, value=1.0, description='a'),
         c_init=widgets.FloatSlider(min=0.5, max=2.0, step=0.1, value=1.0, description='c'),
         delta_init=widgets.FloatSlider(min=0.1, max=1.0, step=0.1, value=0.5, description='delta')
        )


interactive(children=(FloatSlider(value=0.67, description='alpha', max=2.0, min=0.1), FloatSlider(value=1.33, …

interactive(children=(FloatSlider(value=0.67, description='alpha', max=2.0, min=0.1), FloatSlider(value=1.33, …

interactive(children=(FloatSlider(value=3.75, description='r', max=4.5, min=0.1), FloatSlider(value=0.01, desc…

interactive(children=(FloatSlider(value=1.0, description='r', max=3.0, min=0.1), FloatSlider(value=1.0, descri…

interactive(children=(IntSlider(value=3, description='n', max=6, min=2), FloatSlider(value=0.1, description='d…

interactive(children=(FloatSlider(value=1.0, description='r', max=2.0, min=0.5), FloatSlider(value=1.0, descri…

<function __main__.fit_model(r_init=1.0, K_init=1.0, a_init=1.0, c_init=1.0, delta_init=0.5)>